In [1]:
from datetime import datetime
import traceback
import functools
import multiprocessing
from multiprocessing import Pool

import pytz
import time
import os
import sys
from pathlib import Path

import numpy as np
import scipy
import pandas as pd

import matplotlib
matplotlib.use('nbagg')
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from astropy import stats

import astropy.units as u
from astropy.io import fits
from mmtwfs.wfs import WFSFactory

tz = pytz.timezone("America/Phoenix")

In [2]:
def get_traceback(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except Exception as ex:
            ret = '#' * 60
            ret += "\nException caught:"
            ret += "\n"+'-'*60
            ret += "\n" + traceback.format_exc()
            ret += "\n" + '-' * 60
            ret += "\n"+ "#" * 60
            print(sys.stderr, ret)
            sys.stderr.flush()
            raise ex
 
    return wrapper

In [3]:
# instantiate all of the WFS systems...
wfs_keys = ['f9', 'newf9', 'f5', 'mmirs', 'binospec']
wfs_systems = {}
wfs_names = {}
for w in wfs_keys:
    wfs_systems[w] = WFSFactory(wfs=w)
    wfs_names[w] = wfs_systems[w].name
plt.close('all')

# give mmirs a default
wfs_systems['mmirs'].default_mode = 'mmirs1'

# map f9 to oldf9
wfs_systems['oldf9'] = wfs_systems['f9']

In [7]:
def check_image(f, wfskey=None):
    hdr = {}
    with fits.open(f) as hdulist:
        for h in hdulist:
            hdr.update(h.header)
        data = hdulist[-1].data
    
    # if wfskey is None, figure out which WFS from the header info...
    if wfskey is None:
        # check for MMIRS
        if 'WFSNAME' in hdr:
            if 'mmirs' in hdr['WFSNAME']:
                wfskey = 'mmirs'
        if 'mmirs' in f.name:
            wfskey = 'mmirs'

        # check for binospec
        if 'bino' in f.name:
            wfskey = 'binospec'
        if 'ORIGIN' in hdr:
            if 'Binospec' in hdr['ORIGIN']:
                wfskey = 'binospec'

        # check for new F/9
        if 'f9wfs' in f.name:
            wfskey = 'newf9'
        if 'OBSERVER' in hdr:
            if 'F/9 WFS' in hdr['OBSERVER']:
                wfskey = 'newf9'
        if wfskey is None and 'CAMERA' in hdr:
            if 'F/9 WFS' in hdr['CAMERA']:
                wfskey = 'newf9'

        # check for old F/9
        if 'INSTRUME' in hdr:
            if 'Apogee' in hdr['INSTRUME']:
                wfskey = 'oldf9'
        if 'DETECTOR' in hdr:
            if 'Apogee' in hdr['DETECTOR']:
                wfskey = 'oldf9'

        # check for F/5 (hecto)
        if wfskey is None and 'SEC' in hdr:  # mmirs has SEC in header as well and is caught above
            if 'F5' in hdr['SEC']:
                wfskey = 'f5'
        if Path(f.parent / "F5").exists():
            wfskey = 'f5'
            
    if wfskey is None:
        # if wfskey is still None at this point, whinge.
        print(f"Can't determine WFS for {f.name}...")

    if 'AIRMASS' not in hdr:
        if 'SECZ' in hdr:
            hdr['AIRMASS'] = hdr['SECZ']
        else:
            hdr['AIRMASS'] = np.nan

    # we need to fix the headers in all cases to have a proper DATE-OBS entry with
    # properly formatted FITS timestamp.  in the meantime, this hack gets us what we need 
    # for analysis in pandas.
    dtime = None
    if 'DATEOBS' in hdr:
        dateobs = hdr['DATEOBS']
        if 'UT' in hdr:
            ut = hdr['UT'].strip()
        elif 'TIME-OBS' in hdr:
            ut = hdr['TIME-OBS']
        else:
            ut = "07:00:00"  # midnight
        timestring = dateobs + " " + ut + " UTC"
        if '-' in timestring:
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
        else:
            dtime = datetime.strptime(timestring, "%a %b %d %Y %H:%M:%S %Z")

    else:
        if wfskey == "oldf9":
            d = hdr['DATE-OBS']
            if '/' in d:
                day, month, year = d.split('/')
                year = str(int(year) + 1900)
                timestring = year + "-" + month + "-" + day + " " + hdr['TIME-OBS'] + " UTC"
            else:
                timestring = d + " " + hdr['TIME-OBS'] + " UTC"
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
        else:
            if 'DATE-OBS' in hdr:
                timestring = hdr['DATE-OBS'] + " UTC"
                try:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S.%f %Z")
                except:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S %Z")
                # mmirs uses local time in this header pre-2019
                if wfskey == 'mmirs' and dtime < datetime.fromisoformat("2019-01-01T12:00:00"):
                    local_dt = tz.localize(dtime)
                    dtime = local_dt.astimezone(pytz.utc)
            else:
                dt = datetime.fromtimestamp(f.stat().st_ctime)
                local_dt = tz.localize(dt)
                dtime = local_dt.astimezone(pytz.utc)

    if dtime is None:
        print(f"No valid timestamp in header for {f.name}...")
        obstime = None
    else:
        obstime = dtime.isoformat().replace('+00:00', '')
        
    hdr['WFSKEY'] = wfskey
    hdr['OBS-TIME'] = obstime
    return data, hdr

In [13]:
@get_traceback
def process_image(f):
    """
    Process FITS file, f, to get info we want from the header and then analyse it with the 
    appropriate WFS instance. Return results in a comma-separated line that will be collected 
    and saved in a CSV file.
    """            
    data, hdr = check_image(f)
    
    wfskey = hdr['WFSKEY']
    obstime = hdr['OBS-TIME']
    airmass = hdr['AIRMASS']
    exptime = hdr['EXPTIME']
    
    # being conservative here and only using data that has proper slope determination
    # and wavefront solution. also want to get statistics on the quality of the wavefront fits.
    try:
        results = wfs_systems[wfskey].measure_slopes(str(f), plot=False)
    except:
        print(f"Problem analyzing {f.name}...")
        results = {}
        results['slopes'] = None

    if results['slopes'] is not None:
        zresults = wfs_systems[wfskey].fit_wavefront(results, plot=False)
        line = "%s,%s,%s,%f,%f,%f,%f,%f,%f,%f\n" % (
            obstime,
            wfskey,
            f.name,
            exptime,
            airmass,
            results['seeing'].value,
            results['raw_seeing'].value,
            results['fwhm'],
            zresults['zernike_rms'].value,
            zresults['residual_rms'].value
        )
        zfile = f.parent / (f.stem + ".reanalyze.zernike")
        zresults['zernike'].save(filename=zfile)
        spotfile = f.parent / (f.stem + ".spots.csv")
        results['spots'].write(spotfile, overwrite=True)
        return line
    else:
        return None

#rootdir = Path("/Users/tim/MMT/wfsdat")
rootdir = Path("/Volumes/LaCie 8TB/wfsdat")
#rootdir = Path("/Volumes/Seagate2TB/wfsdat")
#rootdir = Path("/mnt/d/wfsdat")

In [15]:
dirs = sorted(list(rootdir.glob("20181*")))  # pathlib, where have you been all my life!
csv_header = "time,wfs,file,airmass,seeing,raw_seeing,fwhm,wavefront_rms,residual_rms\n"
for d in dirs:
    if d.is_dir():
        if Path.exists(d / "reanalyze_results.csv"):
            print("Already processed %s..." % d.name)
        else:
            try:
                lines = []
                lines.append(csv_header)
                night = int(d.name)  # valid WFS directories are ints of the form YYYYMMDD. if not this form, int barfs
                msg = "checking %d... " % night
                fitsfiles = d.glob("*.fits")
                print(msg)
#                 for f in fitsfiles:
#                     print("Processing %s..." % f)
#                     process_image(f)
                nproc = 6 # my mac mini's i7 has 6 cores and py37 can use hyperthreading for more... 
                with Pool(processes=nproc) as pool:  # my mac mini's i7 has 6 cores... 
                    plines = pool.map(process_image, fitsfiles)  # plines comes out in same order as fitslines!
                plines = list(filter(None.__ne__, plines))  # trim out any None entries
                lines.extend(plines)
                with open(d / "reanalyze_results.csv", "w") as f:
                    f.writelines(lines)
            except ValueError as e:  # this means running int(d.name) failed so it's not a valid directory...
                print(f"Skipping %s... ({e})" % d.name)

checking 20181001... 
checking 20181002... 
checking 20181003... 
checking 20181004... 
Already processed 20181005...
checking 20181006... 
checking 20181007... 
checking 20181008... 
checking 20181009... 
checking 20181010... 
checking 20181011... 


Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 43 spots detected out of 108 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 42 spots detected out of 108 apertures.')


checking 20181012... 
checking 20181013... 
checking 20181014... 
checking 20181015... 
checking 20181016... 
checking 20181017... 
checking 20181018... 


Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 1 spots detected out of 110 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 33 spots detected out of 110 apertures.')


checking 20181019... 


...

Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 8 spots detected out of 153 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 15 spots detected out of 153 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 2 spots detected out of 153 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 4 spots detected out of 153 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 5 spots detected out of 153 apertures.')


checking 20181225... 
checking 20181226... 
checking 20181227... 
checking 20181228... 
checking 20181229... 


Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'No WFS spots detected.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 4 spots detected out of 137 apertures.')


checking 20181230... 
checking 20181231... 


Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 1 spots detected out of 137 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 45 spots detected out of 137 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 1 spots detected out of 137 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 63 spots detected out of 137 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 37 spots detected out of 137 apertures.')
Wavefront slope measurement failed: (WFSAnalysisFailed(...), 'Only 25 spots detected out of 137 apertures.')
